In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df_processed = pd.read_csv('../DataBase/merged_dataset.csv')

df_processed = df_processed.dropna(subset=['genre_title']) #EFFACEEEEER

X = df_processed.drop('genre_title', axis=1)
y = df_processed['genre_title']

X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=(0.2 / 0.8), random_state=42, stratify=y_train_val
)

# Random Forest Classifier

model_rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, class_weight='balanced')

model_rf.fit(X_train, y_train) 

y_pred_val = model_rf.predict(X_val)

print("--- Resultados en Conjunto de Validación (RF) ---")
print(classification_report(y_val, y_pred_val))

## Random Forest Classifier

In [ ]:
def prepare_data_for_random_forest(df):
    df = df.dropna(subset=['genre_title'])
    X = df.drop('genre_title', axis=1)
    y = df['genre_title']
    return X, y

In [ ]:
model_rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, class_weight='balanced')

model_rf.fit(X_train, y_train) 

y_pred_val = model_rf.predict(X_val)

print("--- Resultados en Conjunto de Validación (RF) ---")
print(classification_report(y_val, y_pred_val))